In [ ]:
from scapy.all import *
from gevent.queue import Queue
import time, threading, gevent, signal, random

In [ ]:
def keep_sniffing(pckt):
    return False
def old_add_network(pckt, known_networks):
    if not pckt.haslayer(Dot11):
        print(pckt.summary())
    try:
        bssid = pckt[Dot11].addr3
        channel = int(ord(pckt[Dot11Elt:3].info))
        if bssid not in known_networks:
            known_networks[bssid] = (channel,pckt)
            print(bssid+" "+str(channel))
    except IndexError:
        pckt.summary()
    except TypeError:
        pckt.summary()



In [ ]:
network = {}

In [ ]:
def sniff_thread():
        sniff(iface='mon0', lfilter=lambda x: (x.haslayer(Dot11Beacon) or x.haslayer(Dot11ProbeResp)), stop_filter=keep_sniffing, prn=lambda x:add_network(x, network))
    

In [ ]:
t = threading.Thread(target=sniff_thread)
t.setDaemon(True)
t.start()
time.sleep(5)
global stop_sniff
stop_sniff = False

In [ ]:
i = 0
for item in network:
    i = i+1
    print(network[item][1].summary())

In [ ]:
t.is_alive()

In [ ]:
sniff_thread()

In [ ]:
def quick_sniff(q):
    while True:
        sniff(iface='mon0', lfilter=lambda x: (x.haslayer(Dot11) and x[Dot11].type==2), count=20, timeout=0.5, prn=lambda x:q.put(x))
        gevent.sleep(1)

def add_network(q, known_networks):
    while True:
        if not q.empty():
            _add_network(q.get(), known_networks)
        gevent.sleep(0)
        
def _add_network(pckt, known_networks):
    if not pckt.haslayer(Dot11):
        pass
    try:
        bssid = pckt[Dot11].addr3
        if bssid not in known_networks:
            known_networks[bssid] = (None,pckt)
    except (IndexError, TypeError):
        pass

def switch_channel():
    channels = range(1,13)
    while True:
        random.shuffle(channels)
        for channel in channels:
            print('changing channel to '+str(channel))
            subprocess.Popen(['iwconfig', 'mon0', 'channel', str(channel)])
            gevent.sleep(0.5)

In [ ]:
network = {}
q = Queue()
t1 = gevent.spawn(quick_sniff, q)
t2 = gevent.spawn(add_network, q, network)
t3 = gevent.spawn(switch_channel)
ts = [t1, t2]
gevent.signal(signal.SIGINT, gevent.killall, ts)
gevent.joinall(ts, timeout=30)
print("--- END ---")

In [ ]:
[(bssid,network[bssid][0], network[bssid][1].summary()) for bssid in network]

In [ ]:
mac = '00:30:18:ab:89:38'
network[mac][1].show()

In [ ]:
x = network[mac][1][Dot11]

In [ ]:
y = x.FCfield

In [ ]:
y & 0x3 == 0x3

In [ ]:
def packet_dispatch(ppckt):
    ffx = ppckt[Dot11].FCfield & 0x3
    if ffx ^ 0x2 == 0x0:
        packet_10(ppckt)
    elif ffx ^ 0x1 == 0x0:
        packet_01(ppckt)
    elif ffx ^ 0x0 == 0x0:
        packet_00(ppckt)

def packet_10(ppckt1):
    bssid1 = ppckt1[Dot11].addr2
    client1 = ppckt1[Dot11].addr1
    print(bssid1, client1)

def packet_01(ppckt2):
    bssid2 = ppckt2[Dot11].addr1
    client2 = ppckt2[Dot11].addr2
    print(bssid2, client2)
    
def packet_00(ppckt3):
    bssid3 = ppckt3[Dot11].addr3
    client3 = ppckt3[Dot11].addr2
    print(bssid3, client3)

In [ ]:
for b in network:
    packet_dispatch(network[b][1])

In [ ]:
network['18:1b:eb:0f:4e:51'][1][Dot11].addr3

In [ ]:
packet_dispatch(network['00:03:52:b0:22:f0'][1])